### **📝 Client Request: American Option Purchase**

A client has requested that the bank **purchase an American put option** on **Commonwealth Bank of Australia (CBA)** with the following details:

- **Option Type**: American Put  
- **Underlying Asset**: Commonwealth Bank of Australia (CBA)  
- **Strike Price**: $170.00  
- **Expiry Date**: 15 May 2026

To solve this problem, binomial tree method was applied to value the American put option on CBA shares. This model simulates potential future price movements of the stock over discrete time intervals until the option’s expiry on 15 May 2026. Using the Cox-Ross-Rubinstein framework, the model incorporates early exercise features and the time value of money to determine the fair premium the bank should pay for the option with a strike price of $170.00.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import datetime

In [ ]:
def get_CBA_data():
    
    ticker = yf.ticker('CBA.AX')
    hist = ticker.history(start="2025-05-16", end="2025-05-17")

    if hist.empty:
        raise ValueError("No price data available for 16 May 2025.")
    
    S0 = hist['Close'].iloc[0]  # closing price on 16 May 2025

    try:
        info = ticker.info
        dividend_yield = info.get("dividendYield", 0.0) or 0.0  # annual dividend yield
    except:
        dividend_yield = 0.0

    return S0, dividend_yield # returns the closing price and dividend yield

The following function prices an American put option by implementing the Cox-Ross-Rubinstein (CRR) binomial tree model, which accounts for continuous dividend yield.

#### Parameters:

- **S0** (*float*): Spot price of the underlying asset  
- **K** (*float*): Strike price of the option  
- **r** (*float*): Risk-free interest rate (annualised)  
- **q** (*float*): Continuous dividend yield (annualised)  
- **T** (*float*): Time to expiry, expressed in years  
- **sigma** (*float*): Volatility of the underlying asset (annualised)  
- **N** (*int*): Number of steps in the binomial tree  

#### Returns:

- **float**: The calculated premium (price) of the American put option

In [ ]:
def binomial_american_put(S0, K, r, q, T, sigma, N):
    
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp((r - q) * dt) - d) / (u - d)
    disc = np.exp(-r * dt)

    # Build stock tree
    stock = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(i + 1):
            stock[i, j] = S0 * (u ** j) * (d ** (i - j))

    # Build option tree
    option = np.zeros((N + 1, N + 1))
    for j in range(N + 1):
        option[N, j] = max(K - stock[N, j], 0)

  
